In [1]:
import os

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [2]:
!pip install -Uqq --upgrade --force-reinstall langchain langchain-openai

In [3]:
from langchain_openai import ChatOpenAI
import json

model = "gpt-4"

In [4]:
def new_game():
    new_game_prompt = """
    System: You are the game master for a detective game like "where in the world is Carmen San Diego". 
    To start the game, generate the following and return it as a json hash. 
    1. A random uuid, key: case_id
    2. A suspect name: suspect_name
    3. A source city, key: current_city 
    4. A famous artifact from that city, key: stolen_item
    5. A list of 6 unique cities with the current city as the first element, key: hops
    6. Set a field called next_hop to 1
    
    Make the suspect name and artifacts as campy as possible but use real city names. 

    Here is an example:
    {
        case_id: <uuid>,
        suspect_name: <suspect_name>,
        stolen_item: <item name>,
        hops: [
            <city 1>,
            <city 2>,
            <city 3>,
            <city 4>,
            <city 5>,
            <city 6>,
        ],
        next_hop: 1
    Output only the json hash. Do not provide any other text output. 
    """
    llm = ChatOpenAI(api_key=OPENAI_API_KEY, model=model)

    return llm.invoke(new_game_prompt).content
    

In [5]:
new_game()

'{\n    "case_id": "6ba7b810-9dad-11d1-80b4-00c04fd430c8",\n    "suspect_name": "Sneaky Sally Spaghettini",\n    "current_city": "Rome",\n    "stolen_item": "The Colossal Colosseum Columbine",\n    "hops": [\n        "Rome",\n        "Paris",\n        "London",\n        "New York",\n        "Tokyo",\n        "Sydney"\n    ],\n    "next_hop": 1\n}'

In [6]:
game_states = {}

In [7]:
game_state = json.loads(new_game())
game_states[game_state["case_id"]] = game_state
game_states

{'b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b': {'case_id': 'b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b',
  'suspect_name': 'Ruthless Rudy Raffles',
  'current_city': 'Paris',
  'stolen_item': "Mona Lisa's Mischievous Smile",
  'hops': ['Paris', 'Berlin', 'Rome', 'Cairo', 'Moscow', 'New York'],
  'next_hop': 1}}

In [8]:
case_id, _ = next(iter(game_states.items()))
case_id

'b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b'

In [9]:
from langchain_core.tools import tool

@tool
def fetch_game_state(case_id: str) -> str:
    """ Fetch the game state given the case_id """
    return json.dumps(game_states[case_id])

In [10]:
fetch_game_state.invoke({"case_id": case_id})

'{"case_id": "b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b", "suspect_name": "Ruthless Rudy Raffles", "current_city": "Paris", "stolen_item": "Mona Lisa\'s Mischievous Smile", "hops": ["Paris", "Berlin", "Rome", "Cairo", "Moscow", "New York"], "next_hop": 1}'

In [11]:
@tool
def set_current_city(case_id: str, current_city: str):
    """ Sets the current city for the given case_id """
    game_states[case_id]["current_city"] = current_city
    

In [14]:
@tool
def generate_destinations(previous_city: str, next_city: str, current_city: str, exclude_list: str) -> str:
    """ Generates destinations for the player to travel to given the current city, previous city, 
    the next city, and a list of cities to exclude """
    llm = ChatOpenAI(api_key=OPENAI_API_KEY, model=model)

    prompt = f"""
        Generate a list of 4 cities. The list must include {previous_city}, {next_city} 
        and 2 other cities. It must not include {current_city} or any of the
        cities in {exclude_list}. Then shuffle the list before returning it. 
        
        Return this as a json list named destinations. Do not include any additional text.

        Here is an example of the json format:
        destinations: [
            <city 1>,
            <city 2>,
            <city 3>,
            <city 4>,
        ]
        """

    return llm.invoke(prompt).content

In [16]:
generate_destinations.invoke(
    {
        "previous_city": "London",
        "next_city": "Mumbai",
        "current_city": "Tokyo",
        "exclude_list": "Paris,Lima"
    }
)

'{\n    "destinations": [\n        "London",\n        "Mumbai",\n        "New York",\n        "Sydney"\n    ]\n}'

In [58]:
@tool
def generate_regular_clues(city: str) -> str:
    """ Generate regular clues given a location """
    prompt = f"""
        Generate 3 clues. To generate each clue, do the following 
        1. Pick 3 locations at random out of bank, embassy, restaurant, tour guide, library. 
        2. Generate a clue for each of the locations
        
        The clue for a bank should be the currency of the country. For example if the city is London, 
        the clue should say "The suspect changed their currency to Pounds"
        
        The clue for the embassy should be about the flag of the country. For example, if the city is 
        New York, the clue should say "The suspect enquired about visiting a country with a red, white, and blue flag"
        
        The clue for the restaurant should be a good from the country the city is in. For example, for Mumbai, the clue
        should say "The suspect was looking for fresh samosas"
        
        The clue for tour guide should be something about a famous tourist attraction from the city. For example, for 
        Lima the clue should say "The suspect was interested in exploring Incan ruins"
        
        The clue for the library should be about a famous author from the country the city is in. For example, for London
        the clue could say "The suspect was interested in visiting the grave of Charles Dickens"
        
        Return it as a json hash with the following keys as a response:
        1. city: The name of the city
        2. clues: a list of json hashes with keys location and clue
        
        Here is an example of the output json format:
        {{
            "city": <city_name>,
            "clues": [
                {{
                    "location": <location 1>,
                    "clue": <clue 1>,
                }},
                {{
                    "location": <location 2>,
                    "clue": <clue 2>,
                }},
                {{
                    "location": <location 3>,
                    "clue": <clue 3>,
                }},
            ]
        }}
        
        Do not generate any additional text.
       
        Now generate clues for {city}
    """
    return ChatOpenAI(api_key=OPENAI_API_KEY, model=model).invoke(prompt).content

In [59]:
json.loads(generate_regular_clues.invoke(
    {
        "city": "Osaka" 
    }
))

{'city': 'Osaka',
 'clues': [{'location': 'bank',
   'clue': 'The suspect changed their currency to Yen'},
  {'location': 'restaurant',
   'clue': 'The suspect was looking for fresh sushi'},
  {'location': 'tour guide',
   'clue': 'The suspect was interested in visiting Osaka Castle'}]}

In [62]:
@tool
def generate_mistaken_clues(city: str) -> str:
    """ Generate mistaken clues given a location """
    prompt = f"""
        Generate 3 clues. To generate each clue, do the following 
        1. Pick 3 locations at random out of bank, embassy, restaurant, tour guide, library. 
        2. Set the clue to "No one with the suspect's description was seen here"
        
        Return it as a json hash with the following keys as a response:
            a. city: The name of the city
            b. clues: a list of json hashes with keys location and clue

        Here is an example of the output json format:
        {{
            "city": <city_name>,
            "clues": [
                {{
                    "location": <location 1>,
                    "clue": <clue 1>,
                }},
                {{
                    "location": <location 2>,
                    "clue": <clue 2>,
                }},
                {{
                    "location": <location 3>,
                    "clue": <clue 3>,
                }},
            ]
        }}
                
        Do not generate any additional text.

        Now generate clues for {city} 
    """
    return ChatOpenAI(api_key=OPENAI_API_KEY, model=model).invoke(prompt).content

In [63]:
json.loads(generate_mistaken_clues.invoke(
    {
        "city": "Osaka" 
    }
))

{'city': 'Osaka',
 'clues': [{'location': 'bank',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'restaurant',
   'clue': "No one with the suspect's description was seen here"},
  {'location': 'embassy',
   'clue': "No one with the suspect's description was seen here"}]}

In [67]:
@tool
def generate_arrest_clues(city: str, suspect_name) -> str:
    """ Generate arrest clues given a location and the name of the suspect"""
    prompt = f"""
        Generate 3 clues. To generate each clue, do the following 
        1. Pick 3 locations at random out of bank, embassy, restaurant, tour guide, library. 
        2. For two of them set the clue to "Watch your step. You are getting close"
        3. For the third, set the clue to "Congratulations! You have arrested the suspect, {suspect_name}
        
        Return it as a json hash with the following keys as a response:
            a. city: The name of the city
            b. clues: a list of json hashes with keys location and clue

        Here is an example of the output json format:
        {{
            "city": <city_name>,
            "clues": [
                {{
                    "location": <location 1>,
                    "clue": <clue 1>,
                }},
                {{
                    "location": <location 2>,
                    "clue": <clue 2>,
                }},
                {{
                    "location": <location 3>,
                    "clue": <clue 3>,
                }},
            ]
        }}
 
        Do not generate any additional text.

        Now generate clues for {city} 
    """
    return ChatOpenAI(api_key=OPENAI_API_KEY, model=model).invoke(prompt).content

In [69]:
json.loads(generate_arrest_clues.invoke(
    {
        "city": "Osaka",
        "suspect_name": "Stealy McStealy"
    }
))

{'city': 'Osaka',
 'clues': [{'location': 'restaurant',
   'clue': 'Watch your step. You are getting close'},
  {'location': 'library', 'clue': 'Watch your step. You are getting close'},
  {'location': 'embassy',
   'clue': 'Congratulations! You have arrested the suspect, Stealy McStealy'}]}

In [70]:
!pip install -Uqq langchain_experimental

In [71]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=PythonREPL().run,
)

In [72]:
@tool
def update_game_state(case_id, key, value):
    """ Updates the game state based on the values given """
    game_states[case_id][key] = value

In [73]:
tools = [
    fetch_game_state, 
    generate_destinations, 
    generate_regular_clues, 
    generate_arrest_clues, 
    generate_mistaken_clues,
    python_repl,
    update_game_state,   
]

In [74]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

def get_agent_executor():
    prompt = ChatPromptTemplate.from_messages(
        [
          ("system", "You are the game master for a detective game like 'where in the world is Carmen San Diego'. You will have to make a lot of decisions so look at the clues carefully."),
          ("placeholder", "{chat_history}"),
          ("human", "{input}"),
          ("placeholder", "{agent_scratchpad}"),
        ]
    )

    llm = ChatOpenAI(api_key=OPENAI_API_KEY, model=model)
    # Construct the tool calling agent
    agent = create_tool_calling_agent(llm, tools, prompt)

    # Create an agent executor by passing in the agent and tools
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    return agent_executor

In [77]:
def get_destinations(case_id):
    agent_executor = get_agent_executor()
    
    prompt = f"""
    Do the following:
    1. Fetch the game state for case_id {case_id}
    2. Write python code passing in the game state to do the following:
        a. Extract the current_city from the game state obtained above
        b. Get the city at the index next_hop from the hops list in the game state and save it as the next city
        c. If next_hop is 1 then set the previous city to "" otherwise get the city at the index next_hop -1 from 
            the hops list in the game state and save it as the previous city
        d. Add all other cities in the hops list to an exclude list
    
    3. Based on the output above, generate 4 destinations for the player to travel to including the previous city,
    current city, next city, and cities not in the exclude list. 
    4. Shuffle the list
    
    Return the output as a json hash with the following keys:
    1. city: The current city
    2. destinations: a list of the cities

    Here is an example of the output format:
    {{
        "city": <current city>
        "destinations": [
            <city 1>,
            <city 2>,
            <city 3>,
            <city 4>,
        ]
    }}
            
    Generate no additional text 
    """
    ret = agent_executor.invoke(
        {
            "input": prompt
        }
    )

    return ret['output']

In [78]:
json.loads(get_destinations(case_id))



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b'}`


{"case_id": "b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b", "suspect_name": "Ruthless Rudy Raffles", "current_city": "Paris", "stolen_item": "Mona Lisa's Mischievous Smile", "hops": ["Paris", "Berlin", "Rome", "Cairo", "Moscow", "New York"], "next_hop": 1}
Invoking: `python_repl` with `game_state = {'case_id': 'b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b', 'suspect_name': 'Ruthless Rudy Raffles', 'current_city': 'Paris', 'stolen_item': "Mona Lisa's Mischievous Smile", 'hops': ['Paris', 'Berlin', 'Rome', 'Cairo', 'Moscow', 'New York'], 'next_hop': 1}

# Extracting the current_city
current_city = game_state['current_city']

# Get the next city
next_hop_index = game_state['next_hop']
next_city = game_state['hops'][next_hop_index]

# Get the previous city
if next_hop_index == 1:
    previous_city = ''
else:
    previous_city = game_state['hops'][next_hop_index - 1]

# Generat

{'city': 'Paris', 'destinations': ['London', 'Sydney', 'Tokyo', 'Berlin']}

In [79]:
def get_clues(case_id):
    agent_executor = get_agent_executor()
    
    prompt = f"""
    1. Fetch the game state for case_id {case_id}.
    2. Write and execute python code passing in the game state to do the following to figure out what kind of 
       clue to generate:
        a. Extract next_hop from the game state
        b. If next_hop is equal to 5 only then print "arrest clues" as the type of clues
        c. Else If the current city is not in the hops list only then print "mistaken clues" as the type of clue
        d. Else print "regular clues" as the type of clue
    3. Based on the output of the code generate the right type of clue for the city at index next_hop in the hops list
    
    Return the output exactly as a hash encoded in json returned from the tool. Generate no additional text
    """
    ret = agent_executor.invoke(
        {
            "input": prompt
        }
    )

    return ret['output']

In [80]:
json.loads(get_clues(case_id))



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b'}`


{"case_id": "b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b", "suspect_name": "Ruthless Rudy Raffles", "current_city": "Paris", "stolen_item": "Mona Lisa's Mischievous Smile", "hops": ["Paris", "Berlin", "Rome", "Cairo", "Moscow", "New York"], "next_hop": 1}
Invoking: `python_repl` with `game_state = {'case_id': 'b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b', 'suspect_name': 'Ruthless Rudy Raffles', 'current_city': 'Paris', 'stolen_item': 'Mona Lisa's Mischievous Smile', 'hops': ['Paris', 'Berlin', 'Rome', 'Cairo', 'Moscow', 'New York'], 'next_hop': 1}

next_hop = game_state['next_hop']
current_city = game_state['current_city']
hops = game_state['hops']

if next_hop == 5:
    print('arrest clues')
elif current_city not in hops:
    print('mistaken clues')
else:
    print('regular clues')`


SyntaxError('unterminated string literal (detected at line 1)', ('<string>', 1, 257, 

{'city': 'Berlin',
 'clues': [{'location': 'Bank',
   'clue': 'The suspect changed their currency to Euros'},
  {'location': 'Restaurant',
   'clue': 'The suspect was looking for fresh Bratwurst'},
  {'location': 'Library',
   'clue': 'The suspect was interested in the works of Thomas Mann'}]}

In [81]:
cur_city = game_states[case_id]["current_city"]
game_states[case_id]["current_city"] = "Pune"
json.loads(get_clues(case_id))
game_states[case_id]["current_city"] = cur_city



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b'}`


{"case_id": "b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b", "suspect_name": "Ruthless Rudy Raffles", "current_city": "Pune", "stolen_item": "Mona Lisa's Mischievous Smile", "hops": ["Paris", "Berlin", "Rome", "Cairo", "Moscow", "New York"], "next_hop": 1}
Invoking: `python_repl` with `game_state = {'case_id': 'b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b', 'suspect_name': 'Ruthless Rudy Raffles', 'current_city': 'Pune', 'stolen_item': 'Mona Lisa's Mischievous Smile', 'hops': ['Paris', 'Berlin', 'Rome', 'Cairo', 'Moscow', 'New York'], 'next_hop': 1}
next_hop = game_state['next_hop']
if next_hop == 5:
    print('arrest clues')
elif game_state['current_city'] not in game_state['hops']:
    print('mistaken clues')
else:
    print('regular clues')`


SyntaxError('unterminated string literal (detected at line 1)', ('<string>', 1, 256, "game_state = {'case_id': 'b5f8f5b9-7fa6-4f2

In [82]:
nxt_hop = game_states[case_id]["next_hop"]
game_states[case_id]["next_hop"] = 5
json.loads(get_clues(case_id))
game_states[case_id]["next_hop"] = nxt_hop



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b'}`


{"case_id": "b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b", "suspect_name": "Ruthless Rudy Raffles", "current_city": "Paris", "stolen_item": "Mona Lisa's Mischievous Smile", "hops": ["Paris", "Berlin", "Rome", "Cairo", "Moscow", "New York"], "next_hop": 5}
Invoking: `python_repl` with `game_state = {'case_id': 'b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b', 'suspect_name': 'Ruthless Rudy Raffles', 'current_city': 'Paris', 'stolen_item': 'Mona Lisa's Mischievous Smile', 'hops': ['Paris', 'Berlin', 'Rome', 'Cairo', 'Moscow', 'New York'], 'next_hop': 5}
next_hop = game_state['next_hop']
current_city = game_state['current_city']
hops = game_state['hops']
if next_hop == 5:
    print('arrest clues')
elif current_city not in hops:
    print('mistaken clues')
else:
    print('regular clues')`


SyntaxError('unterminated string literal (detected at line 1)', ('<string>', 1, 257, "g

In [92]:
def travel(case_id, city):
    agent_executor = get_agent_executor()
    
    prompt = f"""
        1. Fetch the game state for case_id {case_id}
        2. Write python code passing in the game state to do the following:
            a. If {city} is not in the hops list, update current_city to {city} 
            b. If {city} is in the hops list and the index of {city} is <= next_hop then set next_hop to the index + 1 and current_city to {city}
            c. If the index of {city} is greater than next_hop print error otherwise print out the 
            new values of current_city and next_hop as a json hash

        If there is no error from the python code, return the current city as json hash with a current_city key. 
        If there is an error from the python code, return the error "You cannot travel to that city" as a 
        json hash with the key "error".
        
        Here is an example of the output format when there is no error:
        {{
            "current_city": <city>
        }}

        Here is an example of the output format when there is an error:
        {{
            "error": "You cannot travel to that city"
        }}

        Generate no additional text.

    """

    ret = agent_executor.invoke(
        {
            "input": prompt
        }
    )

    return ret['output']
    

In [94]:
json.loads(travel(case_id, "New York"))



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': 'b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b'}`


{"case_id": "b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b", "suspect_name": "Ruthless Rudy Raffles", "current_city": "Paris", "stolen_item": "Mona Lisa's Mischievous Smile", "hops": ["Paris", "Berlin", "Rome", "Cairo", "Moscow", "New York"], "next_hop": 1}
Invoking: `python_repl` with `game_state = {'case_id': 'b5f8f5b9-7fa6-4f2d-ba86-f0f6db2a5f7b', 'suspect_name': 'Ruthless Rudy Raffles', 'current_city': 'Paris', 'stolen_item': "Mona Lisa's Mischievous Smile", 'hops': ['Paris', 'Berlin', 'Rome', 'Cairo', 'Moscow', 'New York'], 'next_hop': 1};
if 'New York' in game_state['hops']:
    index = game_state['hops'].index('New York');
    if index <= game_state['next_hop']:
        game_state['next_hop'] = index + 1;
        game_state['current_city'] = 'New York';
        print({'current_city': game_state['current_city'], 'next_hop': game_state['next_hop']});
    elif i

{'error': 'You cannot travel to that city'}

In [38]:
game_state

{'case_id': '123e4567-e89b-12d3-a456-426614174000',
 'suspect_name': 'Countess Conundrum',
 'current_city': 'New York',
 'stolen_item': "The Eiffel Tower's Topmost Tip",
 'hops': ['Paris',
  'New York',
  'Rio de Janeiro',
  'Cape Town',
  'Moscow',
  'Tokyo',
  'Sydney'],
 'next_hop': 2}

In [44]:
json.loads(travel(case_id, "Chennai"))
game_state



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Paris", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 1}
Invoking: `python_repl` with `game_state = {"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Paris", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 1}

if 'Chennai' not in game_state['hops']:
    game_state['current_city'] = 'Chennai'
elif 'Chennai' in game_state['hops']:
    index = game_state['hops'].index('Chennai')
    if index <= game_state['next_hop']:
        game_state['next_hop'] = index + 1
        game_state['current_city'

{'case_id': '123e4567-e89b-12d3-a456-426614174000',
 'suspect_name': 'Countess Conundrum',
 'current_city': 'Chennai',
 'stolen_item': "The Eiffel Tower's Topmost Tip",
 'hops': ['Paris',
  'New York',
  'Rio de Janeiro',
  'Cape Town',
  'Moscow',
  'Tokyo',
  'Sydney'],
 'next_hop': 1}

In [45]:
json.loads(get_clues(case_id))



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Chennai", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 1}
Invoking: `python_repl` with `game_state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Countess Conundrum', 'current_city': 'Chennai', 'stolen_item': "The Eiffel Tower's Topmost Tip", 'hops': ['Paris', 'New York', 'Rio de Janeiro', 'Cape Town', 'Moscow', 'Tokyo', 'Sydney'], 'next_hop': 1}
next_hop = game_state['next_hop']
if next_hop == 5:
    print('arrest clues')
elif game_state['current_city'] not in game_state['hops']:
    print('mistaken clues')
else:
    print('regular clues')`


mistaken clues

Invoking: `generate_mistaken_clues` with `{'city': 'Chennai'}`




{'clue1': {'location': 'bank',
  'clue': "No one with the suspect's description was seen here"},
 'clue2': {'location': 'tour guide',
  'clue': "No one with the suspect's description was seen here"},
 'clue3': {'location': 'restaurant',
  'clue': "No one with the suspect's description was seen here"}}

In [46]:
json.loads(travel(case_id, "Paris"))



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Chennai", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 1}
Invoking: `python_repl` with `game_state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Countess Conundrum', 'current_city': 'Chennai', 'stolen_item': "The Eiffel Tower's Topmost Tip", 'hops': ['Paris', 'New York', 'Rio de Janeiro', 'Cape Town', 'Moscow', 'Tokyo', 'Sydney'], 'next_hop': 1}; paris_index = game_state['hops'].index('Paris') if 'Paris' in game_state['hops'] else -1; if paris_index == -1: game_state['current_city'] = 'Paris'; elif paris_index <= game_state['next_hop']: game_state['next_hop'] = paris_index + 1; game_state['current_city'] = 'Paris'; elif pa

{'current_city': 'Paris'}

In [47]:
game_state

{'case_id': '123e4567-e89b-12d3-a456-426614174000',
 'suspect_name': 'Countess Conundrum',
 'current_city': 'Paris',
 'stolen_item': "The Eiffel Tower's Topmost Tip",
 'hops': ['Paris',
  'New York',
  'Rio de Janeiro',
  'Cape Town',
  'Moscow',
  'Tokyo',
  'Sydney'],
 'next_hop': 1}

In [43]:
json.loads(travel(case_id, "Cape Town"))



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Paris", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 1}
Invoking: `python_repl` with `game_state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Countess Conundrum', 'current_city': 'Paris', 'stolen_item': 'The Eiffel Towers Topmost Tip', 'hops': ['Paris', 'New York', 'Rio de Janeiro', 'Cape Town', 'Moscow', 'Tokyo', 'Sydney'], 'next_hop': 1}
if 'Cape Town' in game_state['hops']:
    cape_town_index = game_state['hops'].index('Cape Town')
    if cape_town_index <= game_state['next_hop']:
        game_state['next_hop'] = cape_town_index + 1
        game_state['current_city'] = 'Cape Town'
    elif cape_town_index > game_state

{'current_city': 'Paris'}

In [51]:
game_state = json.loads(new_game())
game_states[game_state["case_id"]] = game_state
game_states

{'123e4567-e89b-12d3-a456-426614174000': {'case_id': '123e4567-e89b-12d3-a456-426614174000',
  'suspect_name': 'Countess Conundrum',
  'current_city': 'Paris',
  'stolen_item': "The Eiffel Tower's Topmost Tip",
  'hops': ['Paris',
   'New York',
   'Rio de Janeiro',
   'Cape Town',
   'Moscow',
   'Tokyo',
   'Sydney'],
  'next_hop': 1},
 '0c4e6e0a-6a71-4d61-a6fc-7c1fcb8cd611': {'case_id': '0c4e6e0a-6a71-4d61-a6fc-7c1fcb8cd611',
  'suspect_name': 'Percival Pickpocket',
  'current_city': 'Paris',
  'stolen_item': "The Louvre's Mona Lisa",
  'hops': ['Paris', 'Tokyo', 'New York', 'Cairo', 'Sydney', 'London'],
  'next_hop': 1}}

In [52]:
case_id, _ = next(iter(game_states.items()))

In [53]:
get_clues(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Paris", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 1}
Invoking: `python_repl` with `game_state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Countess Conundrum', 'current_city': 'Paris', 'stolen_item': 'The Eiffel Tower's Topmost Tip', 'hops': ['Paris', 'New York', 'Rio de Janeiro', 'Cape Town', 'Moscow', 'Tokyo', 'Sydney'], 'next_hop': 1}

next_hop = game_state['next_hop']

if next_hop == 5:
    print('arrest clues')
elif game_state['current_city'] not in game_state['hops']:
    print('mistaken clues')
else:
    print('regular clues')`


SyntaxError('unterminated string literal (detected at line 1)', ('<string>', 1, 278

'{"location": "bank", "clue": "The suspect changed their currency to Dollars"}, {"location": "restaurant", "clue": "The suspect was looking for fresh New York style pizza"}, {"location": "library", "clue": "The suspect was interested in visiting the grave of Edgar Allan Poe"}'

In [54]:
travel(case_id, "New York")



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Paris", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 1}
Invoking: `python_repl` with `game_state = {"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Paris", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 1};

if 'New York' not in game_state['hops']:
    game_state['current_city'] = 'New York'
elif game_state['hops'].index('New York') <= game_state['next_hop']:
    game_state['next_hop'] = game_state['hops'].index('New York') + 1
    game_state['current_city'] = 'New York'
elif game_state[

'{"current_city": "New York"}'

In [55]:
get_clues(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "New York", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 2}
Invoking: `python_repl` with `game_state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Countess Conundrum', 'current_city': 'New York', 'stolen_item': 'The Eiffel Tower's Topmost Tip', 'hops': ['Paris', 'New York', 'Rio de Janeiro', 'Cape Town', 'Moscow', 'Tokyo', 'Sydney'], 'next_hop': 2}
next_hop = game_state['next_hop']
current_city = game_state['current_city']
hops = game_state['hops']
if next_hop == 5:
    print('arrest clues')
elif current_city not in hops:
    print('mistaken clues')
else:
    print('regular clues')`


SyntaxError('unterminated string litera

'{"bank": "The suspect changed their currency to Reais", "restaurant": "The suspect was looking for fresh feijoada", "tour guide": "The suspect was interested in exploring Cristo Redentor"}'

In [56]:
get_destinations(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "New York", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 2}
Invoking: `python_repl` with `game_state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Countess Conundrum', 'current_city': 'New York', 'stolen_item': 'The Eiffel Tower's Topmost Tip', 'hops': ['Paris', 'New York', 'Rio de Janeiro', 'Cape Town', 'Moscow', 'Tokyo', 'Sydney'], 'next_hop': 2}

# Extract the current_city from the game state
current_city = game_state['current_city']

# Get the city at the index next_hop from the hops list in the game state
next_city = game_state['hops'][game_state['next_hop']]

# If next_hop is 1 then set the previous city to '' otherwi

'["New York", "Rio de Janeiro", "Berlin", "Toronto"]'

In [57]:
travel(case_id, "Berlin")



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "New York", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 2}
Invoking: `python_repl` with `game_state = {"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "New York", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 2}

if 'Berlin' not in game_state['hops']:
  game_state['current_city'] = 'Berlin'
else:
  berlin_index = game_state['hops'].index('Berlin')
  if berlin_index <= game_state['next_hop']:
    game_state['next_hop'] = berlin_index + 1
    game_state['current_city'] = 'Berlin'
  elif be

'{"current_city": "Berlin"}'

In [58]:
get_clues(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Berlin", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 2}
Invoking: `python_repl` with `game_state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Countess Conundrum', 'current_city': 'Berlin', 'stolen_item': "The Eiffel Tower's Topmost Tip", 'hops': ['Paris', 'New York', 'Rio de Janeiro', 'Cape Town', 'Moscow', 'Tokyo', 'Sydney'], 'next_hop': 2}
next_hop = game_state['next_hop']
if next_hop == 5:
    print('arrest clues')
elif game_state['current_city'] not in game_state['hops']:
    print('mistaken clues')
else:
    print('regular clues')`


mistaken clues

Invoking: `generate_mistaken_clues` with `{'city': 'Berlin'}`


{
 

'{"clues": [\n  {"location": "bank", "clue": "No one with the suspect\'s description was seen here"},\n  {"location": "embassy", "clue": "No one with the suspect\'s description was seen here"},\n  {"location": "restaurant", "clue": "No one with the suspect\'s description was seen here"}\n]}'

In [59]:
get_destinations(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Berlin", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 2}
Invoking: `python_repl` with `game_state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Countess Conundrum', 'current_city': 'Berlin', 'stolen_item': 'The Eiffel Towers Topmost Tip', 'hops': ['Paris', 'New York', 'Rio de Janeiro', 'Cape Town', 'Moscow', 'Tokyo', 'Sydney'], 'next_hop': 2}
current_city = game_state['current_city']
next_city = game_state['hops'][game_state['next_hop']]
previous_city = '' if game_state['next_hop'] == 1 else game_state['hops'][game_state['next_hop'] - 1]
exclude_list = [city for i, city in enumerate(game_state['hops']) if i not in [game_st

'["Rio de Janeiro", "London", "New York", "Toronto"]'

In [61]:
travel(case_id, "Rio de Janeiro")



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Berlin", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 2}
Invoking: `python_repl` with `game_state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Countess Conundrum', 'current_city': 'Berlin', 'stolen_item': "The Eiffel Tower's Topmost Tip", 'hops': ['Paris', 'New York', 'Rio de Janeiro', 'Cape Town', 'Moscow', 'Tokyo', 'Sydney'], 'next_hop': 2}

if 'Rio de Janeiro' not in game_state['hops']:
    game_state['current_city'] = 'Rio de Janeiro'
    print({'current_city': game_state['current_city'], 'next_hop': game_state['next_hop']})
elif game_state['hops'].index('Rio de Janeiro') <= game_state['next_hop']:
    game_state['nex

'{"current_city": "Rio de Janeiro"}'

In [63]:
get_clues(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Rio de Janeiro", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 3}
Invoking: `python_repl` with `game_state = {"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Rio de Janeiro", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 3}
next_hop = game_state['next_hop']
if next_hop == 5:
    print('arrest clues')
elif game_state['current_city'] not in game_state['hops']:
    print('mistaken clues')
else:
    print('regular clues')`


regular clues

Invoking: `generate_regular_clues` with `{'city': 'C

'{"clues": [{"location": "bank", "clue": "The suspect changed their currency to Rand"}, {"location": "restaurant", "clue": "The suspect was looking for fresh biltong"}, {"location": "library", "clue": "The suspect was interested in the work of Nadine Gordimer"}]}'

In [65]:
get_destinations(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Rio de Janeiro", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 3}
Invoking: `python_repl` with `
game_state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Countess Conundrum', 'current_city': 'Rio de Janeiro', 'stolen_item': 'The Eiffel Tower's Topmost Tip', 'hops': ['Paris', 'New York', 'Rio de Janeiro', 'Cape Town', 'Moscow', 'Tokyo', 'Sydney'], 'next_hop': 3}

current_city = game_state['current_city']
next_hop = game_state['next_hop']
hops = game_state['hops']

next_city = hops[next_hop]

if next_hop == 1:
    previous_city = ''
else:
    previous_city = hops[next_hop - 1]

exclude_list = [city for city in hops if city no

'["Rio de Janeiro", "Cape Town", "London", "Los Angeles"]'

In [66]:
travel(case_id, "Cape Town")



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Rio de Janeiro", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 3}
Invoking: `python_repl` with `game_state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Countess Conundrum', 'current_city': 'Rio de Janeiro', 'stolen_item': "The Eiffel Tower's Topmost Tip", 'hops': ['Paris', 'New York', 'Rio de Janeiro', 'Cape Town', 'Moscow', 'Tokyo', 'Sydney'], 'next_hop': 3}

if 'Cape Town' in game_state['hops']:
    cape_town_index = game_state['hops'].index('Cape Town')
    if cape_town_index <= game_state['next_hop']:
        game_state['next_hop'] = cape_town_index + 1
        game_state['current_city'] = 'Cape Town'
    elif cape_tow

'{"current_city": "Cape Town"}'

In [67]:
get_clues(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Cape Town", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 4}
Invoking: `python_repl` with `game_state = {'case_id': '123e4567-e89b-12d3-a456-426614174000', 'suspect_name': 'Countess Conundrum', 'current_city': 'Cape Town', 'stolen_item': 'The Eiffel Towers Topmost Tip', 'hops': ['Paris', 'New York', 'Rio de Janeiro', 'Cape Town', 'Moscow', 'Tokyo', 'Sydney'], 'next_hop': 4}
next_hop = game_state['next_hop']
if next_hop == 5:
    print('arrest clues')
elif game_state['current_city'] not in game_state['hops']:
    print('mistaken clues')
else:
    print('regular clues')`


regular clues

Invoking: `generate_regular_clues` with `{'city': 'Moscow'}`




'{"clues": [{"location": "bank","clue": "The suspect changed their currency to Rubles"},{"location": "embassy","clue": "The suspect enquired about visiting a country with a white, blue and red flag"},{"location": "restaurant","clue": "The suspect was looking for fresh Borscht"},{"location": "tour guide","clue": "The suspect was interested in exploring the Red Square"},{"location": "library","clue": "The suspect was interested in the works of Fyodor Dostoevsky"}]}'

In [69]:
travel(case_id, "Moscow")



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Cape Town", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 4}
Invoking: `python_repl` with `game_state = {"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Cape Town", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 4}
if 'Moscow' not in game_state['hops']:
    game_state['current_city'] = 'Moscow'
else:
    moscow_index = game_state['hops'].index('Moscow')
    if moscow_index <= game_state['next_hop']:
        game_state['next_hop'] = moscow_index + 1
        game_state['current_city'] = 'Mo

'{"current_city": "Moscow"}'

In [70]:
get_clues(case_id)



> Entering new AgentExecutor chain...

Invoking: `fetch_game_state` with `{'case_id': '123e4567-e89b-12d3-a456-426614174000'}`


{"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Moscow", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 5}
Invoking: `python_repl` with `game_state = {"case_id": "123e4567-e89b-12d3-a456-426614174000", "suspect_name": "Countess Conundrum", "current_city": "Moscow", "stolen_item": "The Eiffel Tower's Topmost Tip", "hops": ["Paris", "New York", "Rio de Janeiro", "Cape Town", "Moscow", "Tokyo", "Sydney"], "next_hop": 5}
next_hop = game_state['next_hop']
current_city = game_state['current_city']
hops = game_state['hops']
if next_hop == 5:
    print('arrest clues')
elif current_city not in hops:
    print('mistaken clues')
else:
    print('regular clues')`


arrest clues

Invoking: `generate_arrest_cl

'{"clues": [\n  {\n    "location": "bank",\n    "clue": "Watch your step. You are getting close"\n  },\n  {\n    "location": "embassy",\n    "clue": "Watch your step. You are getting close"\n  },\n  {\n    "location": "restaurant",\n    "clue": "You have arrested the suspect!"\n  }\n]}'